In [18]:
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import numpy as np


In [19]:
cwd = os.path.abspath(os.getcwd())
urdf_model_path = cwd + "/ur_description/urdf/ur5_robot_simple.urdf"
mesh_dir = cwd + "/ur_description/meshes"

model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_model_path, mesh_dir)

data = model.createData()

In [20]:
for jointName in model.names:
    print(jointName)

universe
shoulder_pan_joint
shoulder_lift_joint
elbow_joint
wrist_1_joint
wrist_2_joint
wrist_3_joint


In [28]:
jointIndex = model.getJointId("wrist_3_joint")
print("wrist 3 joint: ", jointIndex)

wrist 3 joint:  6


In [38]:
q = np.array([0., -1, -1, 0., 0., 0.])
v = np.array([0., 0., 0., 0., 0., 0.])

viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()
viz.display(q)

pin.forwardKinematics(model, data, q)
data.oMi[jointIndex]


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


  R =
 0.416147         0  0.909297
        0         1         0
-0.909297         0  0.416147
  p =  0.15246  0.10915 0.842844

In [24]:
for f in model.frames:
    print(f.name)

universe
root_joint
base_link
shoulder_pan_joint
shoulder_link
shoulder_lift_joint
upper_arm_link
elbow_joint
forearm_link
wrist_1_joint
wrist_1_link
wrist_2_joint
wrist_2_link
wrist_3_joint
wrist_3_link
ee_fixed_joint
ee_link


In [26]:
pin.framesForwardKinematics(model, data, q)
frameIndex = model.getFrameId('ee_link')
data.oMf[frameIndex]


  R =
2.03772e-12   -0.416147    0.909297
          1 4.89664e-12           0
-4.4525e-12    0.909297    0.416147
  p =  0.15246  0.19145 0.842844

In [37]:
# Distance
T_wb = pin.SE3.Identity()
T_wd = pin.SE3.Random()
print(T_wd)
pin.log(T_wb.inverse() * T_wd)


  R =
 0.615572   0.77987  0.113464
-0.259763  0.336718 -0.905066
-0.744039   0.52766  0.409856
  p = -0.302214 -0.871657 -0.959954



  v =  0.347261     -1.33 -0.442958
  w =   1.01155  0.605423 -0.734013

In [42]:
import time
viz.display(q)
N_SIMULATION = 1000
dt = 0.001
for i in range(N_SIMULATION):
    # forward dynamics
    
    # CRBA
    # M = pin.crba(model,data,q)
    # nle = pin.nle(model,data,q,v)
    # dv = -np.linalg.inv(M) @ nle
    
    # ABA
    tau = np.zeros(6)
    dv = pin.aba(model,data,q,v,tau)
    
    v += dv*dt
    q += dt*v + 0.5*dv*dt*dt 

    viz.display(q)
    time.sleep(dt)